import libraries

In [15]:
import os
import joblib as jl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as du
from torch.utils.data import Dataset

read JUND data

In [16]:
class JUND_Dataset(Dataset):
    def __init__(self, data_dir):
        '''load X, y, w, a from data_dir
        convert all to float tensors'''
        super(JUND_Dataset, self).__init__()
        cur_dir = os.path.join(os.getcwd(), data_dir)
        self.X = jl.load(os.path.join(cur_dir, 'shard-0-X.joblib'))
        self.X = torch.tensor(self.X, dtype=torch.float)
        self.y = jl.load(os.path.join(cur_dir, 'shard-0-y.joblib'))
        self.y = torch.tensor(self.y, dtype=torch.float)
        self.w = jl.load(os.path.join(cur_dir, 'shard-0-w.joblib'))
        self.w = torch.tensor(self.w, dtype=torch.float)
        self.a = jl.load(os.path.join(cur_dir, 'shard-0-a.joblib'))
        self.a = torch.tensor(self.a, dtype=torch.float)
        
    def __len__(self):
        '''return len of dataset'''
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        '''return X, y, w, and a values at index idx'''
        return self.X[idx,:], self.y[idx], self.w[idx], self.a[idx]

In [17]:
# create datasets and print basic stats
train_data = JUND_Dataset('train_dataset')
test_data = JUND_Dataset('test_dataset')
val_data = JUND_Dataset('valid_dataset')
print("train X", train_data.X.shape)
print("test X", test_data.X.shape)
print("val X", val_data.X.shape)
print("frac train pos", len(torch.where(train_data.y == 1)[0])/train_data.y.shape[0])
print("frac val pos", len(torch.where(val_data.y == 1)[0])/val_data.y.shape[0])
print("frac test pos", len(torch.where(test_data.y == 1)[0])/test_data.y.shape[0])
print("w_sum", train_data.w.sum(), val_data.w.sum(), test_data.w.sum())

train X torch.Size([276216, 101, 4])
test X torch.Size([34528, 101, 4])
val X torch.Size([34527, 101, 4])
frac train pos 0.004228574738610363
frac val pos 0.004228574738610363
frac test pos 0.0042284522706209455
w_sum tensor(276215.9375) tensor(34526.9961) tensor(34527.9961)


define LSTM model

In [32]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim_lstm, hidden_dim_mlp, use_a=True, dropout=0):
        '''input_dim: input layer dim (# features)
           hidden_dim_lstm: hidden layer dim for LSTM
           hidden_dim_mlp: hidden layer for MLP
           use_a: use accessibility
           dropout: dropout prob'''

        super(LSTM, self).__init__()
        self.dropout = dropout
        self.use_a = use_a
        
        #use batch first
        self.lstm = nn.LSTM(input_dim, hidden_dim_lstm, batch_first=True)

        self.fc1 = nn.Linear(hidden_dim_lstm, hidden_dim_mlp)
        
        if use_a:
            hidden_dim_mlp += 1
        
        self.fc2 = nn.Linear(hidden_dim_mlp, 1)

    def forward(self, x, a):
        # x is Bx101x4, a is Bx1 accessibility values

        #note the last hidden layer state from LSTM
        _, (hn, _) = self.lstm(x)
        
        #remove the first dim to "flatten" the hn state vector
        hn = torch.squeeze(hn, dim=0)
        
        x = self.fc1(hn)

        #add accessibility
        if self.use_a:
            x = torch.cat([x, a], dim=1)

        x = self.fc2(x)

        return x

util function to get final size from CNN (useful when using diff conv, maxpool layers, etc)

In [21]:
def get_sz(model_list, in_val):
    #generate random data of size in_val
    in_dat = torch.rand(*(in_val))
    #run thru layers and get number of elements
    for model in model_list:
        in_dat = model(in_dat).data
    # return number of elements
    return torch.numel(in_dat)

define CNN model

In [19]:
class CNN(nn.Module):
    def __init__(self, seqlen, in_channels, out_dim, hid_channels, kernel_size, 
                 hidden_dim, use_a=True, dropout=0, use_maxpool = False):
        '''seqlen: length of input seq
           in_channels: dim for input seq
           out_dim: output layer dim
           hid_channels: hidden layer channels
           kernel_size: CNN kernel size
           hidden_dim: for hidden layer of MLP
           use_a: append accessibilty
           dropout: use dropout prob
           use_maxpool: use max pooling?
           '''
        super(CNN, self).__init__()
        self.dropout = dropout
        self.use_a = use_a
        self.use_maxpool = use_maxpool
        
        layers = []
        self.conv1 = nn.Conv1d(in_channels, hid_channels, kernel_size)
        layers.append(self.conv1)
    
        if self.use_maxpool:
            self.maxpool1 = nn.MaxPool1d(2) #use kernel_size 2 for maxpool
            layers.append(self.maxpool1)
            
        self.flatten = nn.Flatten()
       
        #figure out final layer size     
        input_sz = (1, in_channels, seqlen)
        lin_dim = get_sz(layers, input_sz)
        
        self.fc1 = nn.Linear(lin_dim, hidden_dim)
        
        #add accessibility
        if use_a:
            hidden_dim += 1
        
        self.fc2 = nn.Linear(hidden_dim, out_dim)
 
    def forward(self, x, a):
        #x is Bx4x101, a is Bx1 accessibility values
        # first swap x axes 1 and 2, since CNN expects channels x seqlen
        #make x  Bx101x4
        x = x.swapaxes(1,2)
        
        x = F.relu(self.conv1(x))
        x = F.dropout(x, p=self.dropout)
        
        if self.use_maxpool:
            x = self.maxpool1(x)
        
        #flatten the last conv layer
        x = self.flatten(x)
       
        # compute hidden layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.dropout)
        
        # concat accessibility
        if self.use_a:
            x = torch.cat([x, a], dim=1)
            
        # compute output layer
        x = self.fc2(x)
        
        return x        

In [20]:
def compute_correct(output, target, weight):
    '''compute the weights for correct prediction
       first apply sigmoid and predict class 1 if >= 0.5, 0 otherwise
    '''
    #use logsigmoid for log space computations
    output = F.logsigmoid(output.detach())
    pred = torch.where(output > F.logsigmoid(torch.tensor(0.5)), 
                       1, 0)

    # add up weights of correct predictions
    correct = torch.sum((pred == target)*weight)
    
    return correct.item()

Evaluation Loop: Used for Validation and Testing

In [22]:
def eval_model(model, data_loader):
    # set model in eval mode, since we are no longer training
    model.eval()
    eval_loss = 0.
    correct = 0.
    
    # turn of gradient computation, will speed up testing
    with torch.no_grad():
        for batch_idx, (data, target, weight, accessibility)\
            in enumerate(data_loader):
            # send batches to device
            data = data.to(device)
            target = target.to(device)
            weight = weight.to(device)
            accessibility = accessibility.to(device)

            # compute forward pass and loss
            output = model(data, accessibility)
            loss = F.binary_cross_entropy_with_logits(
                output, target, weight=weight)

            # sum up batch loss
            eval_loss += loss.item()

            # add up number of correct predictions
            correct += compute_correct(output, target, weight)
            
        # eval loss per example
        eval_loss /= (batch_idx+1)

        # final test accuracy
        eval_acc = correct / data_loader.dataset.w.sum().item()

    #put model back to training mode at end of eval
    model.train()
    return eval_loss, eval_acc

Training Code

In [23]:
def train_model(model):
    for epoch in range(1, epochs + 1):
        sum_loss = 0.
        correct = 0.
        for batch_idx, (data, target, weight, accessibility)\
            in enumerate(train_loader):
            # send batch over to device
            data = data.to(device)
            target = target.to(device)
            weight = weight.to(device)
            accessibility = accessibility.to(device)

            # zero out prev gradients
            optimizer.zero_grad()

            # run the forward pass
            output = model(data, accessibility)
            # compute loss/error with weight per sample
            loss = F.binary_cross_entropy_with_logits(
                    output, target, weight=weight)
            sum_loss += loss.item()

            #compute training accuracy
            correct += compute_correct(output, target, weight)

            # compute gradients and take a step
            loss.backward()
            optimizer.step()

        # average loss per example
        sum_loss /= (batch_idx+1)
        train_acc = correct / train_data.w.sum().item()
        print(f'Epoch: {epoch}, Loss: {sum_loss:.6e}, Acc: {train_acc:.4f}')
        if epoch % 1 == 0:  
            #how does validation do
            val_loss, val_acc = eval_model(model, val_loader)
            print(f'Val Loss: {val_loss:.6e}, Val Acc: {val_acc:.4f}')

Set up training

In [24]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"using device: {device}")

# load data in batches
train_loader = du.DataLoader(dataset=train_data,
                             batch_size=batch_size,
                             shuffle=True)

val_loader = du.DataLoader(dataset=val_data,
                             batch_size=batch_size,
                             shuffle=False)

test_loader = du.DataLoader(dataset=test_data,
                            batch_size=batch_size,
                            shuffle=False)

using device: cuda:0


LSTM model

In [37]:
batch_size = 1000
learning_rate = 0.5
epochs = 5
weight_decay = 0.
dropout = 0.5
hidden_dim_lstm = 128
hidden_dim_mlp = 128
use_a = True

# set model and optimizer
# input is 101x4 array, 4 is in_channels 
# output is 1d since there are 2 classes; use sigmoid

model = LSTM(4, hidden_dim_lstm, hidden_dim_mlp, use_a, dropout)

optimizer = optim.Adam(model.parameters(), lr=learning_rate,
                      weight_decay=weight_decay)

# send model over to device
model = model.to(device)

# train & validate model
train_model(model)

# test model
test_loss, test_acc = eval_model(model, test_loader)
print(f'Test loss: {test_loss:.6e}, accuracy: {test_acc:.4f}')

Epoch: 1, Loss: 4.762763e+01, Acc: 0.5124
Val Loss: 6.602876e-01, Val Acc: 0.5236
Epoch: 2, Loss: 6.273109e-01, Acc: 0.6070
Val Loss: 6.034506e-01, Val Acc: 0.6157
Epoch: 3, Loss: 2.592166e+02, Acc: 0.5480
Val Loss: 5.853099e+00, Val Acc: 0.5000
Epoch: 4, Loss: 2.113618e+01, Acc: 0.5876
Val Loss: 6.763994e-01, Val Acc: 0.6458
Epoch: 5, Loss: 6.384380e-01, Acc: 0.6796
Val Loss: 5.853848e-01, Val Acc: 0.7379
Test loss: 5.757445e-01, accuracy: 0.7316


CNN model

In [44]:
batch_size = 1000
learning_rate = 0.5
epochs = 5
weight_decay = 0.
dropout = 0.5
kernel_size = 5
hid_channels = 32
hidden_dim_mlp = 128
use_a = True
use_maxpool = True

model = CNN(101, 4, 1, hid_channels, kernel_size, hidden_dim_mlp, use_a, dropout, use_maxpool)
    
optimizer = optim.Adam(model.parameters(), lr=learning_rate,
                      weight_decay=weight_decay)


# send model over to device
model = model.to(device)

# train & validate model
train_model(model)

# test model
test_loss, test_acc = eval_model(model, test_loader)
print(f'Test loss: {test_loss:.6e}, accuracy: {test_acc:.4f}')

Epoch: 1, Loss: 1.018251e+01, Acc: 0.5488
Val Loss: 6.237770e-01, Val Acc: 0.5664
Epoch: 2, Loss: 6.069835e-01, Acc: 0.6361
Val Loss: 6.081846e-01, Val Acc: 0.6150
Epoch: 3, Loss: 5.871251e-01, Acc: 0.6601
Val Loss: 5.916642e-01, Val Acc: 0.7172
Epoch: 4, Loss: 5.608146e-01, Acc: 0.6821
Val Loss: 5.552275e-01, Val Acc: 0.7207
Epoch: 5, Loss: 5.480938e-01, Acc: 0.6887
Val Loss: 5.456647e-01, Val Acc: 0.7252
Test loss: 5.331877e-01, accuracy: 0.7286
